In [1]:
import json, os
data = {}
layout_img_folder = '../../layout_new'

## Generate splash style layouts with 8 subfigures

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2

def create_gallery_layout_v3(num_images, canvas_size=(1000, 1000), circle_size=220, max_distance=320):
    """
    Create a gallery layout with non-overlapping circles and resize images to fit in these circles. Ensures that the
    diagonal of the image is smaller than the diameter of the circle and the distance between any two circles does not
    exceed a maximum threshold.

    :param image_sizes: List of tuples containing image width and height.
    :param canvas_size: Size of the canvas (width, height).
    :param min_circle_size: Minimum size of the circles.
    :param max_circle_size: Maximum size of the circles.
    :param max_distance: Maximum allowed distance between any two circles.
    :return: List of circles with their positions and sizes, and resized image dimensions.
    """
    def circles_overlap(circle1, circle2):
        distance = np.sqrt((circle1[0] - circle2[0])**2 + (circle1[1] - circle2[1])**2)
        return distance  < (circle1[2] + circle2[2])/2

    def within_max_distance(circle, other_circles):
        if len(other_circles) == 0:
            return True
        for c in other_circles:
            if np.sqrt((circle[0] - c[0])**2 + (circle[1] - c[1])**2) <= max_distance :
                return True
        return False
    circles = []
    
    
    while True:
        flag = '1'
        for i in range(num_images):
            d = circle_size
            ok_xy = []
            for x in range(d, canvas_size[0] - int(d/2), 5):
                for y in range(d, canvas_size[1] - int(d/2),5):
                    new_circle = (x, y, d)
                    if all(not circles_overlap(new_circle, circle) for circle in circles) and within_max_distance(new_circle, circles[:]):
                        ok_xy.append((x, y))
            if len(ok_xy)>0:
                (x, y) = random.choice(ok_xy)       
                new_circle = (x, y, d)
                circles.append(new_circle)
        
            else:
                print(i)
                flag = '0'
                break
        if flag == '1' and i == num_images - 1:
            break
        

    return circles



def crop_to_circles_with_margin(circles, canvas_size, margin_range=(50, 100)):
    """
    Crop the canvas to only include the part with circles, while keeping a random margin and maintaining the aspect ratio.

    :param circles: List of circles (x, y, d).
    :param canvas_size: Original size of the canvas (width, height).
    :param margin_range: Range for the random margin (min, max).
    :return: Cropped canvas dimensions and offset (x_offset, y_offset, cropped_width, cropped_height).
    """
    # Find the bounding box of all circles
    min_x = min(circle[0] - circle[2]/2 for circle in circles)
    max_x = max(circle[0] + circle[2]/2 for circle in circles)
    min_y = min(circle[1] - circle[2]/2 for circle in circles)
    max_y = max(circle[1] + circle[2]/2 for circle in circles)

    # Add a random margin
    margin = random.randint(*margin_range)
    print(margin)
    min_x = max(min_x - margin, 0)
    max_x = min(max_x + margin, canvas_size[0])
    min_y = max(min_y - margin, 0)
    max_y = min(max_y + margin, canvas_size[1])

    # Calculate the cropped dimensions
    cropped_width = max_x - min_x
    cropped_height = max_y - min_y
    

    return min_x, min_y, cropped_width, cropped_height




In [3]:
layout = []

for num in range(8,9):
    data[num] = []
    for iter in range(2):

        layout_ = {}
        # Example usage with the same image sizes
        circles_v3  = create_gallery_layout_v3(num)
        print(circles_v3)


        # Cropping the layout
        x_offset, y_offset, cropped_width, cropped_height = crop_to_circles_with_margin(circles_v3, (1000, 1000))
        cropped_width = cropped_height = max(cropped_width, cropped_height)
        print(x_offset, y_offset, cropped_width, cropped_height)
        
        output_size = [1000,1000]
        # Create a blank white canvas with the desired output size
        canvas = np.ones((output_size[1], output_size[0], 3), dtype=np.uint8) * 255

        layout_['layout_size'] = canvas.shape
        layout_['layout_id'] = str(num) + '_' + str(iter)
        layout_['pos'] = []
        # # Calculate the scaling factor to fit the cropped layout into the canvas
        scale_factor = min(output_size[0] / cropped_width, output_size[1] / cropped_height)

        # Draw the cropped layout on the canvas
        for (x, y, d)in circles_v3:
            # Calculate the coordinates of the top-left corner of the rectangle
            rect_x = int(x - x_offset - d/2)
            rect_y = int(y - y_offset - d/2)

            
            # Calculate the width and height of the rectangle
            rect_w = int(d/2 * 1.4 * scale_factor)
            rect_h = int(d/2 * 1.4 * scale_factor)

            # Draw the rectangle on the canvas
            cv2.rectangle(canvas, (rect_x, rect_y), (rect_x + rect_w, rect_y + rect_h), (0, 0, 0), 2)
            layout_['pos'].append([rect_x, rect_y, rect_x + rect_w, rect_y + rect_h])
        cv2.imwrite(os.path.join(layout_img_folder, layout_['layout_id']+'.jpg'), canvas)
        layout_['image'] = layout_img_folder + layout_['layout_id']+'.jpg'
        layout.append(layout_)


[(380, 325, 220), (480, 545, 220), (540, 825, 220), (305, 760, 220), (760, 415, 220), (790, 645, 220), (855, 875, 220), (260, 540, 220)]
67
83.0 148.0 917.0 917.0
[(370, 670, 220), (300, 425, 220), (530, 850, 220), (700, 630, 220), (605, 350, 220), (885, 305, 220), (825, 855, 220), (225, 850, 220)]
81
34.0 114.0 966.0 966.0


In [4]:
len(layout)

2

In [5]:
json.dump(layout, open(os.path.join(layout_img_folder, 'layout_new.json'), 'w'))

## Generate grid style layouts with 8 subfigures

In [6]:
import json
import random, math
import cv2
import numpy as np

# Load the intermediate data
intermediate_data = json.load(open(os.path.join(layout_img_folder, 'layout_new.json'), 'r', encoding="utf8"))

# Generate denser layouts
k = '8'
num_of_images = 4
layout = intermediate_data[0]

# Generate denser layouts with aligned edges
for _ in range(num_of_images):
    new_layout = {}
    num_squares = int(k)
    canvas_size = (2048,2048)
    square_size = (120, 120)
    final_canvas_size = (1000, 1000)

    new_layout['layout_id'] = layout['layout_id'].split('_')[0] + '_' + str(int(num_of_images/2) + _)
    new_layout['pos'] = []

    normal_image_edge = layout['pos'][0][2] - layout['pos'][0][0]
    large_image_edge = (layout['pos'][0][2] - layout['pos'][0][0]) * 2
    if _ >= int(num_of_images/2):
        num_column = random.choice(np.arange(int(k)//5+1, min(5,int(k))))
    else:
        num_column = random.choice(np.arange(2,4))
    # Calculate the position of the image in the table
    num_row = math.ceil(int(k)/num_column)
    
    layout_numpy = -1*np.ones((num_row+1, num_column+1))
    # Generate aligned edges
    one_total = 0
    for j in range(int(k)):
        # Calculate the position of the image in the table layout_numpy value==1 means a small box. value==0 means a large box
        row = j // num_column
        col = j % num_column
        if np.random.rand() > 0.5 or _ >= int(num_of_images/2) :
            layout_numpy[row, col] = 1

    one_total = np.sum(layout_numpy == 1)
    # print(layout_numpy)
    if _ < int(num_of_images/2):
        if one_total == 0:
            layout_numpy[row, col] = 1
        elif one_total == int(k):
            layout_numpy[0, 0] = -1
    # print(layout_numpy)
    # Expand the unit with value 0 to a 2x2 area
    num_large_image = 0
    for row in range(layout_numpy.shape[0]):
        if int(k) == np.sum(layout_numpy == 1) + num_large_image:
                break
        for col in range(layout_numpy.shape[1]):
            if int(k) == np.sum(layout_numpy == 1) + num_large_image:
                break
            if layout_numpy[row, col] == -1 and row+1 < layout_numpy.shape[0] and col+1 < layout_numpy.shape[1] \
                and layout_numpy[row+1, col] != 0 and layout_numpy[row, col+1] != 0 and layout_numpy[row+1, col+1] != 0:
                layout_numpy[row, col] = 0
                layout_numpy[row, col+1] = 0
                layout_numpy[row+1, col] = 0
                layout_numpy[row+1, col+1] = 0
                num_large_image += 1
    num_ones_to_move = int(k) - np.sum(layout_numpy == 1) - num_large_image
    

    # Add new columns to accommodate the remaining 1s
    num_new_columns = (num_ones_to_move + num_column - 1) // num_column
    layout_numpy = np.pad(layout_numpy, ((0, 0), (0, num_new_columns)), constant_values=-1)
    # Move the remaining 1s to the new columns
    one_moved = 0
    for row in range(layout_numpy.shape[0]):
        if num_ones_to_move == one_moved:
            break
        for col in range(layout_numpy.shape[1]):
            if num_ones_to_move == one_moved:
                    break
            if layout_numpy[row, col] == -1:
                layout_numpy[row, col] = 1
                one_moved +=1
                
        
    squares = []
    # Update the 'pos' in new_layout
    for row in range(layout_numpy.shape[0]):
        for col in range(layout_numpy.shape[1]):
            
            if layout_numpy[row, col] == 1:
                x = 250 + int(col * normal_image_edge*1.05)
                y = 250 + int(row * normal_image_edge*1.05)
                squares.append([x, y, x + normal_image_edge, y + normal_image_edge])
            elif layout_numpy[row, col] == 0 and row+1 < layout_numpy.shape[0] and col+1 < layout_numpy.shape[1] \
                and layout_numpy[row+1, col] == 0 and layout_numpy[row, col+1] == 0 and layout_numpy[row+1, col+1] == 0:
                x = 250 + int(col * normal_image_edge*1.05)
                y = 250 + int(row * normal_image_edge*1.05)
                layout_numpy[row, col] = -1
                layout_numpy[row, col+1] = -1
                layout_numpy[row+1, col] = -1
                layout_numpy[row+1, col+1] = -1
                squares.append([x, y, x + large_image_edge, y + large_image_edge])

    # Find the bounding box of all squares
    x_min = min([square[0] for square in squares])
    y_min = min([square[1] for square in squares])
    x_max = max([square[2] for square in squares])
    y_max = max([square[3] for square in squares])

    # Add a random margin
    margin = np.random.randint(10, 150)
    x_min -= margin
    y_min -= margin
    x_max += margin
    y_max += margin

    # Calculate the cropped dimensions
    cropped_width = x_max - x_min
    cropped_height = y_max - y_min

    # Calculate the top-right corner of the square
    top_right_corner = (x_max, y_min)

    # Resize canvas and squares with the same ratio
    resize_ratio = 1000 / max(cropped_width, cropped_height)
    resized_squares = [[int((square[0] - x_min) * resize_ratio), int((square[1] - y_min) * resize_ratio),
                        int((square[2] - x_min) * resize_ratio), int((square[3] - y_min) * resize_ratio)]
                        for square in squares]
    resized_canvas = cv2.resize(canvas[y_min:y_max, x_min:x_max], (int(cropped_width * resize_ratio), int(cropped_height * resize_ratio)))

    # Print out the resized squares' location
    print(resized_squares)
    # Draw squares on the canvas
    canvas = np.full((1000, 1000, 3), 255, dtype=np.uint8)  # White canvas
    new_layout['layout_size'] = canvas.shape[:2]
    new_layout['layout_id'] = k + '_' + str(_+int(num_of_images/2))
    new_layout['pos'] = []
    for x, y, x1, y1 in resized_squares:
        cv2.rectangle(canvas, (x, y), (x1, y1), (0, 0, 0), 2)  # Black squares
        new_layout['pos'].append([x, y, x1, y1])

    # Save the resized canvas
    new_layout['layout_id'] = k + '_' + str(_+int(num_of_images/2))
    new_layout['image'] = os.path.join(layout_img_folder, new_layout['layout_id']+'.jpg') 
    cv2.imwrite(new_layout['image'], canvas)
    # cv2.imwrite('test.jpg', canvas)
    # Print out the squares' location in the 1000 by 1000 canvas
    print(resized_squares)
    
    intermediate_data.append(new_layout)



[[52, 52, 396, 396], [413, 52, 585, 224], [594, 52, 767, 224], [775, 52, 947, 224], [413, 232, 585, 405], [52, 413, 224, 585], [232, 413, 577, 757], [52, 594, 224, 767]]
[[52, 52, 396, 396], [413, 52, 585, 224], [594, 52, 767, 224], [775, 52, 947, 224], [413, 232, 585, 405], [52, 413, 224, 585], [232, 413, 577, 757], [52, 594, 224, 767]]
[[42, 42, 218, 218], [226, 42, 579, 394], [597, 42, 773, 218], [781, 42, 957, 218], [42, 226, 218, 402], [597, 226, 773, 402], [42, 411, 218, 587], [226, 411, 579, 763]]
[[42, 42, 218, 218], [226, 42, 579, 394], [597, 42, 773, 218], [781, 42, 957, 218], [42, 226, 218, 402], [597, 226, 773, 402], [42, 411, 218, 587], [226, 411, 579, 763]]
[[150, 150, 376, 376], [387, 150, 612, 376], [623, 150, 849, 376], [150, 387, 376, 612], [387, 387, 612, 612], [623, 387, 849, 612], [150, 623, 376, 849], [387, 623, 612, 849]]
[[150, 150, 376, 376], [387, 150, 612, 376], [623, 150, 849, 376], [150, 387, 376, 612], [387, 387, 612, 612], [623, 387, 849, 612], [150, 623,

In [7]:
intermediate_data

[{'layout_size': [1000, 1000, 3],
  'layout_id': '8_0',
  'pos': [[187, 67, 354, 234],
   [287, 287, 454, 454],
   [347, 567, 514, 734],
   [112, 502, 279, 669],
   [567, 157, 734, 324],
   [597, 387, 764, 554],
   [662, 617, 829, 784],
   [67, 282, 234, 449]],
  'image': '../../layout_new8_0.jpg'},
 {'layout_size': [1000, 1000, 3],
  'layout_id': '8_1',
  'pos': [[226, 446, 385, 605],
   [156, 201, 315, 360],
   [386, 626, 545, 785],
   [556, 406, 715, 565],
   [461, 126, 620, 285],
   [741, 81, 900, 240],
   [681, 631, 840, 790],
   [81, 626, 240, 785]],
  'image': '../../layout_new8_1.jpg'},
 {'layout_id': '8_2',
  'pos': [[52, 52, 396, 396],
   [413, 52, 585, 224],
   [594, 52, 767, 224],
   [775, 52, 947, 224],
   [413, 232, 585, 405],
   [52, 413, 224, 585],
   [232, 413, 577, 757],
   [52, 594, 224, 767]],
  'layout_size': (1000, 1000),
  'image': '../../layout_new/8_2.jpg'},
 {'layout_id': '8_3',
  'pos': [[42, 42, 218, 218],
   [226, 42, 579, 394],
   [597, 42, 773, 218],
   [

In [8]:
# Save the updated data
json.dump(intermediate_data, open(os.path.join(layout_img_folder, 'layout_new.json'), 'w'))


## Generate layouts with 2-7 subfigures from the previous generated layouts

In [9]:
import json
data = json.load(open(os.path.join(layout_img_folder, 'layout_new.json'), 'r', encoding="utf8"))

In [10]:
intermediate_data = {}
intermediate_data['8'] = data

In [11]:
print(len(intermediate_data['8']))

6


In [12]:
random_idex = [[3,0, 6, 2, 7, 4, 1, 5],
               [0,7, 6, 2, 3, 1, 4, 5],
               [0,1,2,6,7,4,3,5],
               [0,2,6,7,1,3,5,4],
               [0,4,2,7,5,6,1,3],
               [0,1,2,3,4,5,6,7],
               ]

In [13]:
random_idex

[[3, 0, 6, 2, 7, 4, 1, 5],
 [0, 7, 6, 2, 3, 1, 4, 5],
 [0, 1, 2, 6, 7, 4, 3, 5],
 [0, 2, 6, 7, 1, 3, 5, 4],
 [0, 4, 2, 7, 5, 6, 1, 3],
 [0, 1, 2, 3, 4, 5, 6, 7]]

In [14]:
new_data = {}
for j in range(9):
    new_data[str(j)] = []
for i in range(len(random_idex)):
    for j in range(8):

        k = '8'
        new_layout = {}
        new_layout['layout_id'] = str(8-j) + '_' + str(i)
        # print(new_layout['layout_id'])
        # print(resized_squares)
        # Draw squares on the canvas
        canvas = np.full((1000, 1000, 3), 255, dtype=np.uint8)  # White canvas
        new_layout['layout_size'] = canvas.shape[:2]
        new_layout['pos'] = []
        new_layout['positional_descriptor'] =[]
        for a in range(len(intermediate_data[k][i]['pos'])-j):
            new_layout['positional_descriptor'].append(new_layout['layout_id']+'_'+str(a))
            new_layout['pos'].append(intermediate_data[k][i]['pos'][random_idex[i][a]])
            [x, y, x1, y1] = intermediate_data[k][i]['pos'][random_idex[i][a]]
            cv2.putText(canvas, str(random_idex[i][a]), (x, y+20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
            cv2.rectangle(canvas, (x, y), (x1, y1), (0, 0, 0), 2)  # Black squares


        # Save the resized canvas
        new_layout['image'] = os.path.join(layout_img_folder, new_layout['layout_id']+'.jpg')
        cv2.imwrite(new_layout['image'], canvas)
        # cv2.imwrite('test.jpg', canvas)
        # Print out the squares' location in the 1000 by 1000 canvas
        new_data[str(8-j)].append(new_layout)


In [15]:
json.dump(new_data, open(os.path.join(layout_img_folder, 'layout_new.json'), 'w', encoding="utf8"), indent='\t', separators=(',', ': '))